Rather than saving landsat tifs to disk then transferring to cloud storage, can we modify lsru to save directly to azure? This would save us from having to montior when an instance fills up.

test azure functions in ecohydro lsru to see if we can save direct to azure, no saving to instance store

In [1]:
import os
from lsru import Usgs
from lsru import Espa
from pprint import pprint
import datetime
import time
import yaml
from azure.storage.file.fileservice import FileService

config_path = "/home/ryan/work/azure_configs.yaml"

with open(config_path) as f:
    configs = yaml.safe_load(f)

# Instantiate Usgs class and login. requires setting config with credentials
usgs = Usgs(conf=configs["download"]["lsru_config"])
usgs.login()
espa = Espa(conf=configs["download"]["lsru_config"])

file_service = FileService(
    configs["storage"]["storage_name"], configs["storage"]["storage_key"]
)

DATA_DIR = configs["storage"]["vm_temp_path"]
WBD_PATH = configs["storage"]["wbd_gdb_path"]

REGION_DIR = os.path.join(
    configs["storage"]["region_dir"], configs["storage"]["region_name"]
)

LANDSAT_DWNLD_DIR = os.path.join(REGION_DIR, "landsat_downloaded")

In [2]:
espa.orders[0].download_all_complete_azure(configs['storage']['container'],configs["storage"]["storage_name"], configs["storage"]["storage_key"] )

LT050320312005101501T1-SC20190411045817.tar.gz skipped. Reason: 'NoneType' object has no attribute 'read'
LT050320312005103101T1-SC20190411010427.tar.gz skipped. Reason: 'NoneType' object has no attribute 'read'
LT050320312005092901T1-SC20190411010314.tar.gz skipped. Reason: 'NoneType' object has no attribute 'read'
LT050320312005071101T1-SC20190411005910.tar.gz skipped. Reason: 'NoneType' object has no attribute 'read'
LT050320312005040601T1-SC20190411005728.tar.gz skipped. Reason: 'NoneType' object has no attribute 'read'
LT050320312005021701T1-SC20190411005603.tar.gz skipped. Reason: 'NoneType' object has no attribute 'read'
LT050320312005082801T1-SC20190411005223.tar.gz skipped. Reason: 'NoneType' object has no attribute 'read'
LT050320312005030501T1-SC20190411004807.tar.gz skipped. Reason: 'NoneType' object has no attribute 'read'
LT050320312005011601T1-SC20190411001516.tar.gz skipped. Reason: 'NoneType' object has no attribute 'read'


### Need to figure out how to use blob storage, try saving and reading landsat tif files

https://github.com/Azure-Samples/storage-blob-python-getting-started

newer version

https://github.com/Azure-Samples/storage-blobs-python-quickstart

In [1]:
import os
import yaml
from azure.storage.blob import BlockBlobService
config_path = "/home/ryan/work/azure_configs.yaml"

with open(config_path) as f:
    configs = yaml.safe_load(f)



# Azure Storage Blob Sample - Demonstrate how to use the Blob Storage service. 
# Blob storage stores unstructured data such as text, binary data, documents or media files. 
# Blobs can be accessed from anywhere in the world via HTTP or HTTPS. 
#
 
# Documentation References: 
#  - What is a Storage Account - http://azure.microsoft.com/en-us/documentation/articles/storage-whatis-account/ 
#  - Getting Started with Blobs - https://azure.microsoft.com/en-us/documentation/articles/storage-python-how-to-use-blob-storage/
#  - Blob Service Concepts - http://msdn.microsoft.com/en-us/library/dd179376.aspx 
#  - Blob Service REST API - http://msdn.microsoft.com/en-us/library/dd135733.aspx 
#  - Blob Service Python API - http://azure.github.io/azure-storage-python/ref/azure.storage.blob.html
#  - Storage Emulator - http://azure.microsoft.com/en-us/documentation/articles/storage-use-emulator/ 
#
 


In [2]:
block_blob_service = BlockBlobService(account_name=configs['storage']['storage_name'], account_key=configs['storage']['storage_key'])

In [6]:
help(block_blob_service.create_blob_from_bytes)

Help on method create_blob_from_bytes in module azure.storage.blob.blockblobservice:

create_blob_from_bytes(container_name, blob_name, blob, index=0, count=None, content_settings=None, metadata=None, validate_content=False, progress_callback=None, max_connections=2, lease_id=None, if_modified_since=None, if_unmodified_since=None, if_match=None, if_none_match=None, timeout=None) method of azure.storage.blob.blockblobservice.BlockBlobService instance
    Creates a new blob from an array of bytes, or updates the content
    of an existing blob, with automatic chunking and progress
    notifications.
    
    :param str container_name:
        Name of existing container.
    :param str blob_name:
        Name of blob to create or update.
    :param bytes blob:
        Content of blob as an array of bytes.
    :param int index:
        Start index in the array of bytes.
    :param int count:
        Number of bytes to upload. Set to None or negative value to upload
        all bytes starti

In [11]:
def run_sample():
    try:
        
        # Create a container called 'quickstartblobs'.
        container_name =configs['storage']['container']
        block_blob_service.create_container(container_name)

        # Set the permission so the blobs are public.
#         block_blob_service.set_container_acl(container_name, public_access=PublicAccess.Container)

        # Create a file in Documents to test the upload and download.
        local_path=os.path.expanduser("~/work/tmp/LT050320312005011601T1-SC20190411001516")
        local_file_name ="LT05_L1TP_032031_20050116_20160913_01_T1_sr_band1.tif"
        full_path_to_file =os.path.join(local_path, local_file_name)


        print("Temp file = " + full_path_to_file)
        print("\nUploading to Blob storage as blob" + local_file_name)

        # Upload the created file, use local_file_name for the blob name
        block_blob_service.create_blob_from_path(container_name, local_file_name, full_path_to_file)

        # List the blobs in the container
        print("\nList blobs in the container")
        generator = block_blob_service.list_blobs(container_name)
        for blob in generator:
            print("\t Blob name: " + blob.name)

    except Exception as e:
        print(e)

In [12]:
run_sample()

Temp file = /home/ryan/work/tmp/LT050320312005011601T1-SC20190411001516/LT05_L1TP_032031_20050116_20160913_01_T1_sr_band1.tif

Uploading to Blob storage as blobLT05_L1TP_032031_20050116_20160913_01_T1_sr_band1.tif

List blobs in the container
	 Blob name: LT05_L1TP_032031_20050116_20160913_01_T1_sr_band1.tif


In [ ]:
        # Clean up resources. This includes the container and the temp files
        block_blob_service.delete_container(container_name)
        os.remove(full_path_to_file)
        os.remove(full_path_to_file2)